In [ ]:
import pandas as pd

In [ ]:
# read the names of companies from .csv (5rd col)
fortune500 = pd.read_csv('sp500_firm_execu.csv')
companies = fortune500.iloc[:,4]

# convert to list of strings and delete duplicates
companies = list(companies)
companies = list(dict.fromkeys(companies))

print(companies)

In [ ]:
companies = companies[2:]

In [ ]:
from datetime import date
from datetime import timedelta
    

class TimeFrame:
    def __init__(self, start: date, end: date):
        self.start = start
        self.end = end

    def __str__(self):
        return f"after:{self.start} before:{self.end}" # google format for dates
    
    def get_path(self):
        return f"from{self.start}to{self.end}"
    

# create a list of TimeFrame objects, every week going back 1 year
timeframes = []
for i in range(0, 52):
    end = date.today() - timedelta(weeks=i*1)
    start = end - timedelta(weeks=1)
    timeframes.append(TimeFrame(start, end))
    
for t in timeframes:
    print(t)


In [ ]:
import requests
from bs4 import BeautifulSoup
import os

TIMEOUT = 5

for company in companies:
    print('Now on company:', company)
    for timeframe in timeframes:
        # Search google for news about the company, save top 10 results for each timeframe
        # the cols should be: date, title, url, snippet
        
        search_query = f'{company} news'
        search_url = f'https://www.google.com/search?q={search_query} {str(timeframe)}&tbm=nws'
        
        try:
            req = requests.get(search_url, timeout=TIMEOUT)
        except:
            print(f"Error with link {search_url}")
        if req.status_code != 200:
            print(f"Error with status code {req.status_code}")
            continue
        
        soup = BeautifulSoup(req.text, 'html5lib')
        
        

        links = soup.select('a:has(div.egMi0)')
        print(len(links))
        links = [link['href'].replace('/url?q=', '').split('&sa')[0] for link in links]
        
        # print the links
        print(links)
        
        
        
        
        dataframes = []
        for link in links:
            
            try:
                req = requests.get(link, timeout=TIMEOUT)
            except:
                print(f"Error with link {link}")
                continue
            
            if req.status_code != 200:
                print(f"Error with status code {req.status_code}")
                continue
            
            article_soup = BeautifulSoup(req.text, 'html5lib')
            
            # get the date
            try:
                article_date = article_soup.select_one('time').text if article_soup.select_one('time') is not None else None
                if article_date is None: 
                    article_date = article_soup.select_one('meta[itemprop="datePublished"]')['content'] if article_soup.select_one('meta[itemprop="datePublished"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="pubdate"]')['content'] if article_soup.select_one('meta[name="pubdate"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="date"]')['content'] if article_soup.select_one('meta[name="date"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="publishdate"]')['content'] if article_soup.select_one('meta[name="publishdate"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="timestamp"]')['content'] if article_soup.select_one('meta[name="timestamp"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="DC.date.issued"]')['content'] if article_soup.select_one('meta[name="DC.date.issued"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="date"]')['content'] if article_soup.select_one('meta[name="date"]') is not None else None
                if article_date is None:
                    #check tags with class publish-date
                    article_date = article_soup.select_one('.publish-date').text if article_soup.select_one('.publish-date') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.date').text if article_soup.select_one('.date') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.timestamp').text if article_soup.select_one('.timestamp') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.pubdate').text if article_soup.select_one('.pubdate') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.date-published').text if article_soup.select_one('.date-published') is not None else None
                if article_date is None:
                    article_date = "Unknown"
            except:
                article_date = "Unknown"
                
            
            # get the title, try title tag first, then h1
            title = article_soup.select_one('title').text if article_soup.select_one('title') is not None else None
            if title is None:
                title = article_soup.select_one('h1').text if article_soup.select_one('h1') is not None else 'Unknown'
                
            # get the snippet from <p> tags
            snippet = ''
            for p in article_soup.select('p'):
                snippet += p.text + ' '
                
            # create a dataframe
            df = pd.DataFrame({'date': [article_date], 'title': [title], 'url': [link], 'snippet': [snippet]})
            dataframes.append(df)
            
            print(df)
        # save in /data/{company}/{timeframe}.csv
        # create dir if not exists
        if len(dataframes) == 0:
            continue
        os.makedirs(f'data/{company}', exist_ok=True)
        df = pd.concat(dataframes)
        
        # make the delimeter a rare symbol like §
        df.to_csv(f'data/{company}/{timeframe.get_path()}.csv', sep='§')
        
 
        

